In [1]:
import pandas as pd
import numpy as np
import scipy as sp 
import matplotlib as plt
import matplotlib.pyplot as pltp
import plotly.plotly as py 
import missingno as msno
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns
from collections import Counter
from __future__ import print_function
%matplotlib inline
pd.set_option('max_rows',300)
pd.set_option('max_columns',200)

In [3]:
xl = pd.ExcelFile('../Data/Complaint_Data_Scraped.xlsx')
sheetnames = xl.sheet_names  # see all sheet names
sheetnames

[u'Panel complaint data scraped',
 u'EIB complaint data scraped',
 u'MICI complaint data scraped',
 u'CAO complaint data scraped',
 u'ADB complaint data scraped']

In [5]:
dfs = {}
for name in sheetnames:
    df = xl.parse(name)
    dfs[name.split()[0]] = df
    print(name.split()[0],df.shape)
    print(df.columns,'\n','*'*40)


Panel (122, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financial Institution',
       u'Project Loan Amount', u'Sector', u'Issues', u'Complaint Status',
       u'Filing Date', u'Registration Start Date', u'Registration End Date',
       u'Eligibility Start Date', u'Eligibility End Date',
       u'Dispute Resolution Start Date', u'Dispute Resolution End Date',
       u'Compliance Review Start Date', u'Compliance Review End Date',
       u'Monitoring Start Date', u'Monitoring End Date', u'Date Closed',
       u'Documents', u'Hyperlink'],
      dtype='object') 
 ****************************************
EIB (197, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financia

** Inital Thoughts**

Why don't they have the same columns, ADB only has 7, MICI has a 'Hyperlink' and 'Hyperlink [Correct]', need to check that this is unaltered output from the scrapers. 

Other thoughts for doing comparison
1. Do we want them to be aggregated into a single file for comparison?  or just compare to the output files per scraper ? 
2. At the DataDive should we have the teams just work with one scrapers output, and then a fake set of altered data ... this might make more sense. 
3. Need to figure out how to create fake changed data. 

**Check On Shared Total **

In [21]:
def shared_columns(df1,df2):
    col1 = set(df1.columns.tolist())
    col2 = set(df2.columns.tolist())
    
    print( "Shared Cols: ", col1 & col2)
    return list(col1 & col2)

def add_some_prefix(df,prefix,dupe_cols):
    df.columns = [prefix +c if c not in dupe_cols else c for c in df.columns]
    return df


def shared_summary(df_list, print_per_col=True):
    file_cols = []
    for df in df_list:
        cols = df_list[df].columns.tolist()
        for c in cols:
            file_cols.append((df,c))
    shared = pd.DataFrame(file_cols,columns=['colname','table'])
    shared = shared.groupby('table').agg(lambda x: list(x))
    shared['cnt'] = shared.colname.apply(len)
#     shared = shared[shared.cnt > 1]
    shared = shared.sort_values('cnt',ascending=False)
    if print_per_col:
        for i in shared.index:
            print ( 'Col: ',i,'||', len(shared))
            for v in shared.loc[i].values[0]:
                print (' '*10 + '--> ' + v)
    return shared


In [24]:
shared = shared_summary(dfs, print_per_col=False)

In [25]:
shared

,colname,cnt
table,,
Project Number,"[ADB, EIB, CAO, MICI, Panel]",5
Country,"[ADB, EIB, CAO, MICI, Panel]",5
IAM,"[ADB, EIB, CAO, MICI, Panel]",5
Sector,"[ADB, EIB, CAO, MICI, Panel]",5
Related Project Number,"[EIB, CAO, MICI, Panel]",4
Registration Start Date,"[EIB, CAO, MICI, Panel]",4
Registration End Date,"[EIB, CAO, MICI, Panel]",4
Project Type,"[EIB, CAO, MICI, Panel]",4
Compliance Review End Date,"[EIB, CAO, MICI, Panel]",4
